# Data stages
01_raw <- Aligned and labeled TPA+RGB. Not handled by the project. 
02_intermediate <- Aligned and labeled TPA+RGB that was cropped. Crop parameters can vary. Not handled by the project.
03_procesed <- Intermediate cropped data after normalization. Input to the models.